# Building an Image Classifier

Author: Nathan Robertson

A core underlying hypothesis of this project is: **if** we can classify the luxury of a Zillow listing based on how it looks, **then** we will build a significantly better predicter, **because** images tell a story that text data can't. A 3 bed - 2 bath ranch can look like a dump, or it can look like a million dollar home. A human being can look at a series of images and quickly make this determination, but a human can't look at _millions_ of images. So we need to find a way to train a convolutional neural network that can do this work for us.

As this is a graduate school portfolio project, time and funding are limited. I can't label these pictures all by hand, nor can I pay others to do it for me. So instead, we're going to try leveraging off the shelf AI tools to help generate the training dataset. From there, we will build a convolutional neural network from scratch with two layers. The first layer will predict what type of room the picture is (example: bedroom, bathroom, living room), and the second layer will be a series of neural networks for each room type to rank its perceived quality (with a 10 being outstanding quality, and a 1 being horrible quality).

There are convolutional neural networks out there that could handle the first stage - but this is a graduate school project, so let's have fun and build this pizza from scratch. Besides: the second layer (a room type-specific determination of quality on a 1-10 scale) is not something that exists off the shelf, so we'd still need to build that.

The end output of this will be a series of 50,000 - 100,000 photos with predicted room types and quality. These will be used to train the neural network that classifies the 6~ million photos in the data set.

### Step 0: Import packages.

Import packages that will handle.

* Data manipulation.
* Interacting with OpenAI's API.
* Visualizing and extracting images.

In [1]:
# For data manipulation.
import pandas as pd
import ast

# AI for autogenerating image labels.
import openai
client = openai.OpenAI(api_key='')

# Visualizing images
from IPython.display import display, HTML

# Handling image extraction.
from PIL import Image
import requests
from io import BytesIO
import os

# Handle saving images
import random
import string

# For progress tracking.
from tqdm import tqdm
import time

# Connecting to Microsoft Azure.
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

key = ""
endpoint = ""

### Step 1: Melt data into new DataFrame

Let's take the existing data from Zillow webscraping, and read it in. We'll then isolate the list of images for each listing, and melt that into a new DataFrame with one row for each unique image.

In [2]:
# Read in data.
df = pd.read_csv('BACKUP zillow_listing_data.csv')

/var/folders/0l/38yb605j6z30lbrmlsfhrq0h0000gn/T/ipykernel_63489/1142338921.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/backup/BACKUP zillow_listing_data.csv')


In [3]:
# How many photos are there?

count = 0

for i, row in df.iterrows():
    # If there is at least one photo.
    try:
        # Count each photo.
        for photo in ast.literal_eval(row['photosList']):
            count += 1
    # If there are no photos, skip.
    except:
        pass

print('# of photos in dataset:', count)

# of photos in dataset: 6138344


In [4]:
"""
melt_photo_data

Turn the original webscraped Zillow DataFrame into a melted two column DataFrame of images.

    Args:
      already_generated: if True, skip the function and just load the data from a prior run.
      
    Returns:
      A melted DataFrame of Zillow images, including the Zillow listing ID the photo came from
      and the URL of where the photo lives on Zillow's servers.

"""

def melt_photo_data(already_generated=False):

    if already_generated == False:
        # Set the chunk size
        chunk_size = 1000
        
        # Empty lists for capturing results.
        zillowIds = []
        photos = []
        
        # Chunk and process the data
        for chunk_start in tqdm(range(0, df.shape[0], chunk_size), desc="Processing chunks"):
            
            chunk_end = min(chunk_start + chunk_size, df.shape[0])
            chunk_df = df.iloc[chunk_start:chunk_end]
        
            for i, row in chunk_df.iterrows():
                # If there is at least one photo.
                try:
                    # Count each photo.
                    for photo in ast.literal_eval(row['photosList']):
                        zillowIds.append(row['zillowId'])
                        photos.append(photo)
                        
                # If there are no photos, skip.
                except:
                    pass
        
        # Create a two-column DataFrame of all zillowId / photo URL combinations.
        photos_df = pd.DataFrame(data={
            'zillowId': zillowIds,
            'photo': photos
        })
        
        photos_df.to_csv('BACKUP listing_photos_dataset.csv', index=False)
    
        return photos_df

    elif already_generated == True:
        return pd.read_csv('BACKUP listing_photos_dataset.csv', index_col=False)

In [5]:
# Create the DataFrame.
photos_df = melt_photo_data(already_generated=True)

Taking a look at the results, we can see one row for each image. The attributes are the Zillow listing ID from where the photo came, and a URL to where that photo lives.

In [6]:
# Take a look at the results.
photos_df.head()

,zillowId,photo
0,2091638544,https://photos.zillowstatic.com/fp/7ea447eb095...
1,2091638544,https://photos.zillowstatic.com/fp/de4b606d805...
2,2091638544,https://photos.zillowstatic.com/fp/ac0607cb7db...
3,2091638544,https://photos.zillowstatic.com/fp/aec8b02cdf3...
4,2091638544,https://photos.zillowstatic.com/fp/02b2a5d3d0a...


In [7]:
photos_df['photo'][0]

'https://photos.zillowstatic.com/fp/7ea447eb0954cacc829420178ead5fe4-p_d.jpg'

### Step 2: Create automated image labeler

I attempted to label images myself. Using Streamlit, I got into a pretty good workflow...but I couldn't label more than 500 images an hour. Time is limited, and I unfortunately don't have the bandwidth to label these images with the human care I'd like. But, labeling a thousand or so images gave me a decent sense for what type of rules I might pass along to a more automated solution.

So instead, we're going to leverage ChatGPT and Microsoft Azure Vision AI. With a little bit of elbow grease on the prompt engineering, we can get something that is a satisfactory quality up and running that can generate room label and room quality rankings for a little about $0.01 USD / image.

**The flow**

* First, `azure.ai.vision.imagenaalysis` will generate a description of the image.
* Then `gpt-4o` will look at the description and determine what type of room it is from a list of options I created.
* Using `gpt-4o`, and some of the listing metadata where appropriate, we're going to convert that description into a 1-10 numerical rating of the quality of the listing.

### Step 2.1 Define ChatGPT / Azure AI API connections

We're going to create the 3 functions we will use to interact with ChatGPT and Azure AI to classify the room, describe its quality, and generate a ranking.

The first one is fairly straightforward. We're going to pass a list of potential room types into a prompt. `gpt-4-vision-preview`'s job is to determine which room type it thinks best represents the image.

In [8]:
"""
azure_describe_room

Look at an image, and generate a description of it.

    Args:
        url: The url to the Zillow listing photo being assessed.
        
    Returns:
        A result object dictionary that contains all of the information about the picture.
"""

def azure_describe_room(url):
    # Create an Image Analysis client
    client = ImageAnalysisClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key)
    )
    
    # Extract the visual features from the url.
    result = client.analyze_from_url(
        image_url=url,
        visual_features=[VisualFeatures.TAGS,
            VisualFeatures.CAPTION,
            VisualFeatures.DENSE_CAPTIONS
            ]
    )
    
    # Return the result.
    return result

In [9]:
"""
unpack_azure_results

Look at a description generated by azure_describe_room, and pull out the relevant information
into a condensed set of text that can be fed to ChatGPT.

    Args:
        result: the result object generated by the function `azure_describe_room`.
    
    Returns:
        A string object with the minimal information required to pass onto ChatGPT.
"""

def unpack_azure_results(result):
    
    # Placeholder strings.
    caption = '[]'
    denseCaption = '[]'
    tags = '[]'

    # For each string, attempt to extract the information. If it fails for any reason, pass.
    # Passing means the empty value of '[]' will be returned for that portion of the final string.
    try:
        caption = result['captionResult']['text']
    except:
        pass
    try:
        denseCaption = '; '.join([denseCaption['text'] for denseCaption in result['denseCaptionsResult']['values']])
    except:
        pass
    try:
        tags = ', '.join([tag['name'] for tag in result['tagsResult']['values']])
    except:
        pass
    
    # Build final string.
    full_description = f"""*caption* {caption}. *denseCaptions* {denseCaption}. *tags* {tags}."""

    # Return final string.
    return full_description

In [10]:
"""
label_room

Look at an image, and determine what type of room it is.

    Args:
      full_description: The Azure AI generated description returned in `unpack_azure_result`.
      
    Returns:
      The predicted label, as well as the prompt and completion tokens generated by
      the prompt (the latter two will be used to track the estimated cost of the image label).

"""

def label_room(full_description):

    # The options from which to choose between.
    room_options = [
        "Bedroom","Living Room","Kitchen","Bathroom","Dining Room","Home Office","Studio","Loft",
        "Entertainment Room","Laundry","Hallway", "Yard","Garden","Patio","Balcony","Terrace",
        "Pool","Deck","Location Exterior","Garage", "Waterfront View","City View","Mountain View",
        "Countryside View","Forest","Other"
    ]
    
    # What if you made the option list tighter?
    room_options = [
        "Bedroom","Living Room","Kitchen","Bathroom","Location Exterior","Other"
    ]

     # Make 5 attempts to complete the call.
    for attempt in range(1,6):
        try:
            # Determine which label best fits the image.            
            completion = client.chat.completions.create(
              model="gpt-4o",
              messages=[
                {"role": "system", "content": "Pick the option from this list that best describes the picture. You must pick an option from this list. Only pick “Location Exterior” if it looks like the home structure itself is included in the picture. If you are unsure which option to pick, return Other. List: {}".format(room_options)},
                {"role": "user", "content": "Picture description: {}".format(full_description)}
              ]
            )
        
            # save the label, and the tokens required to generate the response.
            label = completion.choices[0].message.content
            completion_tokens = int(completion.usage.completion_tokens)
            prompt_tokens = int(completion.usage.prompt_tokens)
                    
            # return values.
            return label, completion_tokens, prompt_tokens

         # If exception, try again (up to 5 times).
        except Exception as e:
            print('label room error')
            print(e)
            pass
    
    # If 5 attempts fail, return Nones.
    return None, 0, 0
    

_This might be deemed redundant now that Microsoft Azure Vision AI is integrated into the solution._

The next function gets a little more creative with its prompt engineering. We have a dictionary of prompts for each room type, with slightly different instructions for each one tailored to the room in question.

In [11]:
"""
describe_room

Look at an image, and describe its quality.

    Args:
      url: The url to the Zillow listing photo being assessed.
      label: The room type gpt labeled the photo as.
      
    Returns:
      A 60~ word quality description, as well as the prompt and completion tokens generated by
      the prompt (the latter two will be used to track the estimated cost of the image label).

"""

def gpt_describe_room(url, label):

    # Don't waste time if you sense a failure upstream.
    if label == None:
        return None, 0, 0
    
    # Dictionary of prompts for each label type.
    quality_prompts = {
        'Bedroom': "In 60 words or less, describe how the quality of the bedroom of a home on Zillow in the provided picture looks to you. Consider factors such as lighting, space, age of the room, size of room, and overall comfort. Your description should make it clear if this image portrays a low, low-medium, medium, medium-high, high, or very high value home.",
        'Living Room': "In 60 words or less, describe how the quality of the living room of a home on Zillow in the provided picture looks to you. Take into account elements like furniture, lighting, age of the room, size of room, and overall ambiance. Your description should make it clear if this image portrays a low, low-medium, medium, medium-high, high, or very high value home.",
        'Kitchen': "In 60 words or less, describe how the quality of the kitchen of a home on Zillow in the provided picture looks to you. Consider factors such as appliances, storage, age of the room, size of the room, and finishes. Your description should make it clear if this image portrays a low, low-medium, medium, medium-high, high, or very high value home.",
        'Bathroom': "In 60 words or less, describe how the quality of the bathroom of a home on Zillow in the provided picture looks to you. Take into consideration cleanliness, fixtures, age of the room, size of the room, finishes,  and overall design. Your description should make it clear if this image portrays a low, low-medium, medium, medium-high, high, or very high value home.",
        'Location Exterior': "In 60 words or less, describe how the quality of the exterior location of a home on Zillow in the provided picture looks to you.  Consider factors such as curb appeal, landscaping, size, how well maintained the building appears, and overall exterior aesthetics. Your description should make it clear if this image portrays a low, low-medium, medium, medium-high, high, or very high value home."
    }    
    
    # If label is other, don't attempt to describe the quality! Saves a lot of utilization.
    if label == 'Other':
        return None, 0, 0
    
    # Select the appropriate prompt.
    selected_prompt = quality_prompts[label]

    # Make 5 attempts to complete the call.
    for attempt in range(1,6):
        try:
            # Describe the quality of the labeled image.
            response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
              {
                  "role": "user",
                  "content": [
                    {"type": "text", "text": selected_prompt},
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": url,
                        "detail": "high"
                      },
                    },
                  ],
                }
              ],
              max_tokens=100,
            )
            # save the description, and the tokens required to generate the response.
            description = response.choices[0].message.content
            completion_tokens = int(response.usage.completion_tokens)
            prompt_tokens = int(response.usage.prompt_tokens)
            
            #print(description)
        
            # return values
            return description, completion_tokens, prompt_tokens

        # If exception, try again (up to 5 times).
        except Exception as e:
            print('describe room error')
            print(e)
            pass

    # If 5 attempts fail, return Nones.
    return None, 0, 0

The final one converts the quality description into  a 1-10 ranking. This one uses `gpt4` since we're dealing with a text to text artificial intelligence task.

In [12]:
"""
rank_room

Look at an image, and describe its quality.

    Args:
      full_description: The gpt generated description of the room.
      label: The room type gpt labeled the photo as.
      
    Returns:
      A quality rating (1-10 scale), as well as the prompt and completion tokens generated by
      the prompt (the latter two will be used to track the estimated cost of the image label).

"""

def rank_room(full_description, label):

    # Don't waste time if you sense a failure upstream.
    if full_description == None or label == None:
        return None, 0, 0
    
    
    # Dictionary of prompts for each label type.
    rating_prompts = {
    'Bedroom': "On a scale of 1-10, rate the quality of this bedroom based on this text description. Consider factors such as lighting, space, age of the room, size of the room, and overall comfort and perceived value of the home. Only impressive rooms of high or very high value get an 8 or higher. You must return, and only return, an integer between 1 and 10.",
    'Living Room': "On a scale of 1-10, rate the quality of this living room based on this text description. Take into account elements like furniture, lighting, age of the room, size of the room, and overall ambiance and perceived value of the home. Only impressive rooms of high or very high value get an 8 or higher. You must return, and only return, an integer between 1 and 10.",
    'Kitchen': "On a scale of 1-10, rate the quality of this kitchen based on this text description. Consider factors such as appliances, storage, age of the room, size of the room, finishes, and perceived value of the home. Only impressive rooms of high or very high value get an 8 or higher. You must return, and only return, an integer between 1 and 10.",
    'Bathroom': "On a scale of 1-10, rate the quality of this bathroom based on this text description. Take into consideration cleanliness, fixtures, age of the room, size of the room, finishes, and overall design and perceived value of the home. Only impressive rooms of high or very high value get an 8 or higher. You must return, and only return, an integer between 1 and 10.",
    'Location Exterior': "On a scale of 1-10, rate the quality of this exterior location based on this text description. Consider factors such as curb appeal, landscaping, how well maintained the building appears, size, and overall exterior aesthetics and perceived value of the home. Only impressive exteriors of high or very high value get an 8 or higher. You must return, and only return, an integer between 1 and 10."
    }
    
    if label == 'Other':
        return None, 0, 0

    # Selec the appropriate prompt.
    selected_prompt = rating_prompts[label] + ' Description: ' + full_description
    
    # Make 5 attempts to complete the call.
    for attempt in range(1,6):
        try:
            completion = client.chat.completions.create(
              model="gpt-4o",
              messages=[
                {"role": "system", "content": "You determine quality of different photos of a property based on a text description generated by Microsoft Azure Vision. You are biased toward giving lower scores -- don't give away high scores for just anything! If a home seems shabby, dated, or quaint - this is a lower score. If it seems luxurious, high end -- this is a higher score. Just return an integer between 1 and 10 based on the text description provided - nothing else. You must complete the task."},
                {"role": "user", "content": selected_prompt}
              ]
            )

            # save the rating, and the tokens required to generate the response.
            rating = int(completion.choices[0].message.content)
            completion_tokens = int(completion.usage.completion_tokens)
            prompt_tokens = int(completion.usage.prompt_tokens)
        
            # return values
            return rating, completion_tokens, prompt_tokens
            
        # If exception, try again (up to 5 times).
        except Exception as e:
            print('rank room error')
            print(e)
            time.sleep(60)
            pass

    # If 5 attempts fail, return Nones.
    return None, 0, 0

### Step 2.2: Track spend

OpenAI provides tools on their website to track spend -- but I think it is also helpful to keep track of it here. Using the price data available, this function takes in the prompt and completion tokens (or, said another way, the input and output tokens) and determines the cost of the API call.

In [13]:
"""
calculate_spend

Determine how much money was spent on an API call.

    Args:
      completion: the number of tokens outputted by ChatGPT to complete the task.
      prompt: th enumber of tokens inputted to instruct ChatGPT on the task.
      
    Returns:
      The cost in USD of the completion and prompt tokens given to the prompt.

"""

def calculate_spend(completion, prompt):
    
    # Renaming the variables to the naming used in a billing context.
    input_ = prompt
    output = completion

    # Input (prompt) cost: $2.50 / 1000000 tokens.
    # Output (prompt) cost: $10.00 / 1000000 tokens.
    cost = (input_ / 1000000 * 2.50) + (output / 1000000 * 10.00)

    # Return cost.
    return cost
    

### Step 2.3: Standardize, transform, and save image.

Looking ahead to the Convolutional Neural Network(s) that I will train, I would like to have these images available locally to reference as I build the model. I was also take the time transform the images to make uniformly sized. We'll use shrinking and padding to accomplish this.

First, we'll transform the image.

In [14]:
"""
transform_image

Take a Zillow listing image and convert it into a uniformly sized image.

    Args:
      url: The url where a photo resides.
      target_size: the pixel length x width size of the image after it is transformed.
      
    Returns:
      A resized, padded, and cropped photo that matches the target_size.
"""

def transform_image(url, target_size=150):
    try:
        # Download the image from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for unsuccessful responses
        image_data = BytesIO(response.content)

        # Open the image using Pillow
        image = Image.open(image_data)

        # Calculate the target size and the ratio
        target_width, target_height = (target_size, target_size)
        width_ratio = target_width / image.width
        height_ratio = target_height / image.height

        # Determine the new dimensions after resizing and cropping
        if width_ratio < height_ratio:
            new_width = target_width
            new_height = int(image.height * width_ratio)
        else:
            new_width = int(image.width * height_ratio)
            new_height = target_height

        # Resize the image to fit within the new dimensions while maintaining the aspect ratio
        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Randomly crop the resized image to the target size
        left = random.randint(0, max(0, new_width - target_width))
        top = random.randint(0, max(0, new_height - target_height))
        right = left + target_width
        bottom = top + target_height

        cropped_image = resized_image.crop((left, top, right, bottom))

        return cropped_image

    except Exception as e:
        print('transform_image',e)
        return None

Then, we'll create a function for downloading that image for reference later.

In [15]:
"""
download_image

Save the image locally, and return its path.

    Args:
      image: the transformed image.
      
    Returns:
      image_path: the path where the image was saved locally.
"""

def download_image(image):
    try:
        # generate image name.
        characters = string.ascii_letters + string.digits
        image_name = ''.join(random.choice(characters) for _ in range(32))

        # Save the transformed image in the "images" folder
        image_path = os.path.join("data/labeled_images/2 - images", f"""{image_name}.png""")
        image.save(image_path)

        return image_path

    except Exception as e:
        print('download_image',e)
        return None

### Step 3: Build GPT Classifier for image labeling

Let's now start to string together all of the pieces into one function.

In [16]:
"""
gpt_image_classifier

Walks the full path from a Zillow listing photo to its predicted classification.

    Args:
      row: A pandas Series generated from a DataFrame.iterrows() loop.
      
    Returns:
      A pandas DataFrame with the Zillow listing photo, its prediction, the cost to 
      label it, and a location to the transformed image.

"""

def gpt_image_classifier(row):

    # Pull values out of row.
    url = row['photo']
    zillowId = row['zillowId']
    
    #print(zillowId, url)

    # Track spend.
    completion_tokens = 0
    prompt_tokens = 0
    
    # Generate Azure description.
    azure_result = azure_describe_room(url=url)
    azure_description = unpack_azure_results(result=azure_result)
    
    # Generate label.
    label, completion, prompt = label_room(full_description=azure_description)
    completion_tokens += completion
    prompt_tokens += prompt
    
    # Generate description.
    gpt_description, completion, prompt = gpt_describe_room(url=url, label=label)
    completion_tokens += completion
    prompt_tokens += prompt
    
    # Generate ranking, 1-10, using gpt_description.
    rank, completion, prompt = rank_room(full_description=gpt_description, label=label)
    completion_tokens += completion
    prompt_tokens += prompt
    
    # Calculate cost of classification.
    cost = calculate_spend(completion=completion_tokens, prompt=prompt_tokens)

    # Save image for reference.
    image = transform_image(url=url)
    image_path = download_image(image=image)
    
    # Put into pandas dataframe.
    df = pd.DataFrame(data={
        'zillowId':[zillowId],
        'url':[url],
        'image_path':[image_path],
        'label':[label],
        'rank':[rank],
        'gpt_description':[gpt_description],
        'azure_description': [azure_result],
        'cost':[cost],
        'completion_tokens':[completion_tokens],
        'prompt_tokens':[prompt_tokens]
    },index=[0])

    # Return results
    return df

With the ability to now process one photo end to end, let's set it up so we can do that at scale...

In [17]:
"""
save_data_to_csv

A helper function to save data as it is processed.

    Args:
      data: The labeled image data.
      existing_data: Whether or not there is existing data to append this to.
      filename: The name of the file that should be created to store the data.
      
    Returns:
      The DataFrame in its current state with the images processed.
"""

def save_data_to_csv(data, existing_data=None, filename='gpt_image_labels.csv'):

    # If a pandas DataFrame is not passed through for existing_data, just save the data.
    if isinstance(existing_data, pd.DataFrame) == False:
        data.to_csv(filename, index=False)

    # If there is existing data, append `data` to the existing data, and save it.
    elif isinstance(existing_data, pd.DataFrame) == True:
        data = pd.concat([existing_data, data])
        data.to_csv(filename, index=False)
    
    # Either way -- whatever is in this csv is now the source of truth. Read it in as
    # `existing_data, to which future loops will append.
    existing_data = pd.read_csv(filename, index_col=False)

    return existing_data

In [18]:
"""
classify_images

The function that will loop through a set number of photos, saving results intermittently.

    Args:
      numberPictures: the number of pictures we want to pay ChatGPT to label.
      chunkSize: how many photos at a time should we process before saving results.
      df: the DataFrame that we will fetch our images from (this will just be `photos_df`).
      existing_data: Is there an existing DataFrame to work from? (Helpful if the function is interrupted).
      
    Returns:
      The DataFrame of results.
"""

def classify_images(numberPictures, chunkSize, df=photos_df, existing_data=None):

    # For capturing the values.
    imageDataList = []

    # For each row in the provided DataFrame, filtered for the number of pictures we wish to tag.
    for i, row in tqdm(df.sample(frac=1)[:numberPictures].iterrows(), total=numberPictures, desc='Generating image labels'):
        try:
            # Classify image
            one_image = gpt_image_classifier(row)
            
            #print('one_image')
            #print(one_image)

            # Save transformed image.
            imageDataList.append(one_image)
        
        # Sometimes one of the 3 GPT prompts might fail. That's okay.
        # We'll just move on and try the next image.
        except Exception as e:
            print(e)
            pass

        # Only call this block in intervals that matches every chunk size.
        if i % chunkSize == 0:
            
            # Concatenate all the data so far, and save it. If there is existing data, it will concatenate with that.
            if len(imageDataList) > 0:
                data = pd.concat(imageDataList)
                existing_data = save_data_to_csv(data=data, existing_data = existing_data)

            # Clear out the list since we've already written the data from this chunk.
            imageDataList.clear()

    # Finish up writing data for any values that weren't included in the final chunk (ex: if there are
    # 120 pictures with a chunkSize of 50, this would capture the last 20.
    if len(imageDataList) > 0:
        data = pd.concat(imageDataList)
        existing_data = save_data_to_csv(data=data, existing_data = existing_data)

    return existing_data
    

### Step 4: Run it!

Based on some testing done, I expect 10,000 images will cost about $100 (+/- $10 USD). That is about as much as I'd like to spend for a project like this, as I can always bolster the dataset with manual tagging. But given that I can tag about 500 photos in an hour, this represents 20~ hours of work I no longer have to do (So I am paying ChatGPT about $5 USD per hour of my life back). I can live with that!

In [ ]:
gpt_image_labels = classify_images(numberPictures=60000, chunkSize=1000)

In [ ]:
gpt_image_labels.head(50)